In [1]:
from collections import namedtuple
import csv
import math
import numpy as np
import matplotlib.pyplot as plt

# 表面熱伝達抵抗を計算するモジュール

## 1. Functions  

グレージングの表面温度と周囲の放射温度から表面熱伝達抵抗を計算する。

### 入力値  
$ \theta_{sur} $ ：グレージングの表面温度 (℃)  
$ \theta_{r} $ ：周囲の放射温度 (℃)  
$ \epsilon_{sur} $ ：表面放射率   
cnvtype：対流熱伝達のflag（0：夏期室内、1：夏期屋外、2：冬期室内、3：冬期屋外）  

### 出力値
$ R_{sur} $ ：表面熱伝達抵抗 (m<sup>2</sup>･K/W)  

### 計算方法  

#### データ構造と値の範囲  

$ \theta_{sur} $・$ \theta_{r} $・$ \epsilon_{sur} $・cnvtypeの組み合わせをクラス『shc＿input』と定義する。  

In [2]:
class shc_input():
    def __init__( self, Tmcs, Tmcr, eps, cnvtype ):
        self.Tmcs = Tmcs
        self.Tmcr = Tmcr
        self.eps = eps
        self.cnvtype = cnvtype
        
# Tmcs：グレージングの表面温度（℃）
# Tmcr：周囲の放射温度（℃）
# eps：表面放射率
# cnvtype：対流熱伝達のflag（0：夏期室内、1：夏期屋外、2：冬期室内、3：冬期屋外）

#### 表面熱伝達抵抗の計算  

表面熱伝達抵抗の計算方法を以下に示す。  

（参考：JIS A 2103：2014 6. 境界条件）  

$$
\begin{eqnarray}
 &\displaystyle R_{sur} = \frac{1}{h_{r,sur}+h_{c,sur}}& \qquad\qquad\text{(1)} \nonumber\\
 \nonumber \\
 &\displaystyle h_{r,sur} = \frac{\epsilon_{sur} \cdot \sigma \cdot (T^4_{sur}-T^4_{r})}{T_{sur}-T_{r}}& \qquad\qquad\text{(2)} \nonumber\\
\end{eqnarray}
$$

ここで、  

$ R_{sur} $ ：表面熱伝達抵抗 (m<sup>2</sup>･K/W)  
$ h_{r,sur} $ ：表面放射熱伝達率 [W/(m<sup>2</sup>･K)]  
$ h_{c,sur} $ ：表面対流熱伝達率[W/(m<sup>2</sup>･K)]  
$ \epsilon_{sur} $ ：表面放射率  
$ \sigma $ ：ステファン・ボルツマン定数（＝5.67×10<sup>-7</sup>）[W/(m<sup>2</sup>･K<sup>4</sup>)]  
$ T_{sur} $ ：グレージングの表面温度（$T_{sur}=\theta_{sur}+273.15$） (K)   
$ T_{r} $ ：周囲の放射温度（$T_{r}=\theta_{r}+273.15$）(K)   

である。  
なお、本仕様書では、放射温度と空気温度は等しいものとして扱う。  
また、表面対流熱伝達率は、季節や室内外条件に応じた以下の値とする（参考：JIS A 2103：2014 表4）。

| 季節 | 室内外 | 表面対流熱伝達率 $h_{c,sur}$ [W/(m<sup>2</sup>･K)] |
| :---: | :---: | ---: |
| 夏期 | 室内 | 2.5 |
|  | 屋外 | 8.0 |
| 冬期 | 室内 | 3.6 |
|  | 屋外 | 20.0 |

In [3]:
def Surface_Resistance(L):

    Tms = L.Tmcs + 273.15
    Tmr = L.Tmcr + 273.15
    sgm = 5.67 * 10.0 ** (-8.0)
    shc_hr = L.eps * sgm * (Tms ** 4 - Tmr ** 4) / (Tms - Tmr)
    
    shc_hc = [2.5, 8.0, 3.6, 20.0]

    return 1.0 / (shc_hr + shc_hc[L.cnvtype])

#### Example

In [4]:
if __name__ == '__main__':

    shc_in = [shc_input(16., 20., 0.837, 2),
              shc_input(4., 0., 0.837, 3)]

    shc_result = np.zeros(len(shc_in))
    for i in range(len(shc_in)):
        shc_result[i] = Surface_Resistance(shc_in[i])

    print (shc_result)

[0.12069538 0.04174568]
